## __1. 종류__
---

### __1.1 이진 분류기__
데이터를 양성 혹은 음성 2가지로 분류하는 알고리즘입니다.  
이진 분류기는 데이터가 양성에 들어갈 점수(확률)을 구하고 어떤 <b>임계값</b>보다 높으면 양성으로 분류합니다.  

### __1.2 다중 분류기__
여러개의 클래스로 분류할 수 있는 알고리즘입니다.  
다중분류는 알고리즘 자체에서 지원하기도 하지만, 이진분류기의 조합으로 구현될 수 있습니다.  
> __OvR/OvA 전략__ (One versus Rest/All)  
    각각 하나의 클래스인지 분류하는 이진분류기를 클래스 수만큼 만들어 가장 높은 점수로 분류합니다.

> __OvO 전략__ (One versus One)  
    한 클래스와 다른 한 클래스를 분류하는 이진분류기 $n\choose k$개 만들어 다수결로 분류합니다.  
    작지만 많은 훈련을 하는게 유리한 svm등 몇몇이 사용합니다.

### __1.3 다중레이블 분류__
레이블, 즉 목표값이 여러 개인 이진분류기입니다.  
다중 레이블 분류를 지원하는 알고리즘으로 동일하게 시행하며, 성능평가는 각각의 레이블 별 평가를 하고 중요도에 따라 계산합니다. 
> `sklean'에서는 'average='weighted'`로 설정하면 지지도(각 레이블별 샘플 수)로 가중치를 줄 수 있습니다.

### __1.4 다중출력 분류__
목표값이 여러개이면서 클래스 또한 2개 이상으로 분류하는 알고리즘입니다.


## __2. 평가지표__
---

### __2.1 오차행렬__  
실제값과 예측값을 비교하여 정확하게 분류하였는지를 나타내는 표입니다.  
보통 분류의 평가지표는 해당 값들의 계산으로 이루어져 있습니다.

||Positive|Negative|
|---|---|---|
|__Positive__|True Positive|False Positive|
|__Negative__|False Negative|True Negative|

<br>  

* __정확도__, accuracy $=({TP+TN\over Entire})$  
    정확하게 분류한 정도를 나타내는 지표입니다.  
    하지만 불균형한 데이터 세트에서는 전부 하나로 분류해도 높은 성능이 나올 수 있기 때문에 조심해야 합니다.

* __정밀도__, precision $=({TP\over TP+FP})$  
    양성으로 분류한 것중에 진짜 양성의 비율입니다.  

* __재현율__, recall $=({TP\over TP+FN})$  
    진짜 양성 중 제대로 예측한 양성의 비율입니다.

* __F1 Score__ $=2*{precision * recall \over precision + recall}$  
    정밀도와 재현율의 조화평균으로, 두 지표의 값이 비슷하면 높게 나옵니다.  
    정밀도와 재현율은 둘 다 높이기 힘들기 때문에, 두 지표 중에 더 중요한 지표가 없다면 F1을 사용하기 좋습니다.  

### __2.2 정밀도/재현율 트레이드 오프__
임계값이 높아지면 재현율은 줄지만 정밀도가 올라가기 때문에 임계값을 조정하여 정밀도와 재현율을 조정할 수 있습니다. 

> `sklearn.metrics.precision_recall_curve`로 임계값에 따른 정밀도/재현율을 구할 수 있습니다.
```python
    #method='decision_function', 'proba'등을 통해 예측결과가 아닌 점수(확률)를 가져옵니다.
    score = sklearn.model_selection.cross_val_predict(MODEL, X, Y, cv=N, 
        method='decision_function')
    precisions, recalls, threshholds = precision_recall_curve(Y, score)
    #_.metrics.PrecisionRecallDisplay()를 통해 바로 시각화 할 수 있습니다.
```

### __2.3 ROC 곡선__
<b>FP Rate</b>$({FP\over FP+TN})$에 대한 <b>재현율(TP Rate)</b>의 곡선입니다.  
ROC곡선 또한 임계값을 조정하여 트레이트오프할 수 있습니다.  

> TN비율을 <b>특이도</b>라고 하며 FPR은 1-특이도로 구할 수도 있습니다.  

> `sklean.metrics.roc_curve()`, `plot_roc_curve()` 등의 함수가 있습니다.  

주로 ROC곡선 아래의 면적 <b>AUC</b>를 구하여 넓을수록 좋은 모델로 모델 비교를 합니다. 

> `sklean.metrics.roc_auc_score()`

## __3. 에러 분석__
---

분류에서 모델의 성능이 안좋게 나올경우, 오차행렬을 통해 분석해 볼 수 있습니다.
> `sklearn.metrics.confusion_matrix(y_true,y_pred)`

오차행렬의 수치를 보아도 괜찮지만, 시각화하는 것이 직관적입니다.  
> `sklearn.metrics.plot_confusion_matrix(), matplotlib.matshow()`

정확한 분석을 위해 에러 개수를 비율로 변환한 뒤, 주대각선을 0으로 바꾸면 됩니다.
```python
    confusion_matrix = confusion_matrix/confusion_matrix.sum(axis=1,keepdims=True)
    numpy.fill_diagonal(confusion_matrix,0)
```